In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import zipfile

from shapely.geometry import LineString
import networkx as nx
import itertools

Read in the GTFS from a .zip file

In [3]:
gtfs = {}

with zipfile.ZipFile("example-data/septa-bus-gtfs.zip", "r") as z:
    for filename in z.namelist():
        if filename.endswith("txt"):
            with z.open(filename) as f:
                gtfs[filename[:-4]] = pd.read_csv(f)

In [3]:
gtfs.keys()

dict_keys(['agency', 'calendar', 'calendar_dates', 'fare_attributes', 'fare_rules', 'routes', 'shapes', 'stop_times', 'stops', 'transfers', 'trips'])

## How to make a network

Transit networks have special characteristics deviating from normal networks (for example, the street network). They include:

### Variable edge weights by schedule and real-time factors

It is straightforward to consider "edges" in a transit network as the rides between stops, and the weights of these edges are the associated travel times. However, travel times may vary

- By schedule: the GTFS dictates the schedule of bus speeds, which may vary by time of day, day of week, or other factors.

- Actual traffic situations: Even if a GTFS schedule dictates certain travel speeds, it **may not** be worthwhile to stringently follow scheduled departure/arrival times, as the actual travel speeds are dependent on many other factors. Therefore, sometimes it might be enough to just used more simplified travel times for certain periods, especially for more frequent transit services. The errors or deviation from using this approach from the actual condition are not necessarily larger than estimations using exact schedules.

### Problems with transfers

When building the transit network in a more traditional way, we may consider "stops" to be nodes, and "rides" to be edges. A typical path finding algorithm may want to find the shortest path between nodes by minimizing the sum of edge weights. However, there are two problems with this approach:

- In most cases, riders avoid transfers as much as possible, and for most "nodes" in a path, the rider would simply continue on the same trip, without even making the decision whether to switch. The critical decisions in path finding are only made at a small number of nodes.

- Transfers are not "weightless". They take time for the next available bus to arrive.

### Trade-off between accuracy, computation complexity, and schedule disruptions

One possible solution to the above problems is to build a "time-expanded" network: rather than considering each geographic stop as a node, we consider each *event* of departure or arrival at each stop across the day as separate **nodes**, and **edges** include not just rides between stops, but also the stopping events and transfers that happen between stops. Theoretically, this could simulate the operation of the transit system in the most accurate way. However, this approach also has the following problems:

- Computation complexity. The number of nodes and edges in the network would be much larger than the original network, and the computation complexity would be much higher. This is especially true when we are not just calculating for occasional path finding, but for a large number of OD pairs to measure the connectivity of the entire network.
- Schedule disruptions. Real-time disruptions may render all the extral computational work not worthwhile, as real-time conditions often cause discrepancies between the scheduled and actual arrival/departure events, which are not necessarily smaller than the discrepancies from using a simplified estimation method (i.e., using a fixed travel speed or headway for a given period).

### Missing connections between nearby stops

Regardless if we consider "stops" as nodes or "events" as nodes, some connections always tend to be missed, that is, the connections between stops where no riding connections are available, but accessible by walking under certain minutes.

## A Hybrid Approach

Trading off the benefits and drawbacks of different network-building techniques, we propose a hybrid approach:

**Two Networks**: we build both a time-expanded network, which considers each event as a node, and a simplified space-expanded network (maybe two for morning and evening peaks). The time-expanded network is used for the path finding between single or a small number of OD pairs with exact departure times. The space-expanded network is used for the path finding between a large number of OD pairs, for which the departure times are not necessarily exact. The latter approach is used to measure the connectivity of the entire network or between city districts by averaging travel times between a large number of OD pairs.

**Walking Connections**: apart from "ride" edges, and "wait" and "transfer" edges for the time-expanded network, we add additional "walk" edges between stops that are within a certain walking distance. This is to ensure that the path finding algorithm can find the shortest path between any two stops, even if there is no riding connection between them.

## Step 1: Making Walking Connections

In [4]:
stops_gdf = gpd.GeoDataFrame(
    gtfs["stops"],
    geometry=gpd.points_from_xy(gtfs["stops"].stop_lon, gtfs["stops"].stop_lat),
    crs="EPSG:4326"
).to_crs("EPSG:2272")

In [5]:
quarter_mile = 0.25 * 5280

In [6]:
results = []

for index, stop in stops_gdf.iterrows():
    buffer = stop.geometry.buffer(quarter_mile)

    # Use the spatial index to find stops within the buffer
    potential_matches_index = list(stops_gdf.sindex.intersection(buffer.bounds))
    potential_matches = stops_gdf.iloc[potential_matches_index]

    # Check actual distances
    for _, potential_stop in potential_matches.iterrows():
        if (
            stop["stop_id"] != potential_stop["stop_id"]
            and stop["geometry"].distance(potential_stop["geometry"]) <= quarter_mile
        ):
            results.append(
                {
                    "from_stop_id": stop["stop_id"],
                    "to_stop_id": potential_stop["stop_id"],
                    "distance": stop["geometry"].distance(potential_stop["geometry"]),
                }
            )
walk_connections = pd.DataFrame(results)

# Consider a walking speed of 4 feet per second, and suppose the actual routing distance is 1.5 times the straight-line distance.
walk_connections["time"] = walk_connections["distance"] * 1.5 / 4

## Step 2: Make a Time-Expanded Network

**Preparation**: Convert string arrival_time and departure_time to timedelta (because we do not care about dates for now)

In [7]:
# Convert arrival_time and departure_time to timedelta
def cast_time(time_col):
    overflow_mask = time_col.str.slice(0, 2).astype(int) >= 24
    overflows = pd.Series(np.maximum(time_col.str.slice(0, 2).astype(int) - 24, 0))
    result_col = time_col.copy()
    result_col.loc[overflow_mask] = overflows.astype(str).str.zfill(2) + time_col.str.slice(
        2,
    )
    result_col = pd.to_timedelta(result_col)

    result_col.loc[overflow_mask] += pd.to_timedelta(1, unit="d")

    return result_col

In [8]:
stop_times = gtfs["stop_times"].copy()

stop_times["arrival_time"] = cast_time(stop_times["arrival_time"])
stop_times["departure_time"] = cast_time(stop_times["departure_time"])

Now we have a table `stop_times` with each **row** representing a **ride** between two stops. The columns include:

- TRIP_ID
- stop_id
- stop_sequence
- arrival_time: in timedelta
- departure_time: in timedelta

The rows are sorted **first by TRIP_ID, then by stop_sequence**.

In [9]:
# Sort by TRIP and STOP_SEQUENCE
stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,350890,0 days 08:39:00,0 days 08:39:00,20965,1
1,350890,0 days 08:43:00,0 days 08:43:00,82,2
2,350890,0 days 08:47:00,0 days 08:47:00,140,6
3,350890,0 days 08:52:00,0 days 08:52:00,20966,11
4,350890,0 days 08:54:00,0 days 08:54:00,1279,15


**Start making the network:**

### 1. **Add all the nodes**

Each node should have a unique ID. The IDs are formatted in the following way:
`stop_id-trip_id-type`.

- `stop_id` is the stop_id of the stop where the event happens
- `trip_id` is the trip_id of the trip where the event happens
- `type` is either `arrival` or `departure`
  
Iterrate through each row in the `stop_times` table. **Each row has two nodes**, one departure and another arrival.

In [35]:
G1 = nx.Graph()

for _, event in stop_times.iterrows():
    # Arrival time
    G1.add_node(
        f"{event['stop_id']}-{event['trip_id']}-arrival",
        type="arrival",
        time=event["arrival_time"],
    )
    # Departure time
    G1.add_node(
        f"{event['stop_id']}-{event['trip_id']}-departure",
        type="departure",
        time=event["departure_time"],
    )

In [47]:
G1.number_of_nodes() # 4,088,584

4088584

### **2. Add "Ride" edges and "Wait" edges**

A "ride" edges is between the departure node of one stop to the arrival node of the next stop in the same trip.
A "wait" edges is between the arrival node of one stop to the departure node of the same stop in the same trip.

To add the "ride" and "wait" edges, we first group the `stop_times` table by trip. For each trip, all the rows are stacked up in order.
A "Ride" edge is formed from one row to the next, and a "Wait" edge is formed within a single row.

In [36]:
# Save so that we don't ruin the original
G_ride_wait_edges = G1.copy()

In [37]:
for trip_id, group in stop_times.groupby("trip_id"):
    # Generate event IDs for each event
    group["arrival_event"] = (
        group["stop_id"].astype(str)
        + "-"
        + group["trip_id"].astype(str)
        + "-"
        + "arrival"
    )
    group["departure_event"] = (
        group["stop_id"].astype(str)
        + "-"
        + group["trip_id"].astype(str)
        + "-"
        + "departure"
    )

    for i in range(len(group) - 1):
        # Add "Ride" edges
        # From the "departure time" of the previous stop to the "arrival time" of the next stop
        G_ride_wait_edges.add_edge(
            group.iloc[i]["departure_event"],
            group.iloc[i + 1]["arrival_event"],
            type="ride",
            weight=group.iloc[i + 1]["arrival_time"] - group.iloc[i]["departure_time"],
        )

    for i in range(len(group)):
        # Add "Wait" edges
        G_ride_wait_edges.add_edge(
            group.iloc[i]["arrival_event"],
            group.iloc[i]["departure_event"],
            type="wait",
            weight=group.iloc[i]["departure_time"] - group.iloc[i]["arrival_time"],
        )

In [93]:
G_ride_wait_edges.number_of_nodes() # 4,088,584

# The number of nodes stays the same, meaning it's correct

4088584

In [95]:
nx.write_adjlist(G_ride_wait_edges, "data/G_ride_wait_edges.adjlist")

In [10]:
G_ride_wait_edges = nx.read_adjlist("data/G_ride_wait_edges.adjlist")

In [11]:
G_transfer_edges = G_ride_wait_edges.copy()

3. Add "Transfer" edges

In [12]:
# First, identify only the stops where more than one route stops,
# Transfer edges can only happen within these stops

trip_route_dict = gtfs["trips"][["route_id", "trip_id"]]
stop_times_with_route = stop_times.merge(
    trip_route_dict, left_on="trip_id", right_on="trip_id", how="left"
)

stops_route_count = stop_times_with_route.groupby("stop_id")["route_id"].nunique()
transfer_stops = stops_route_count[stops_route_count > 1].index

# Mask for stop_times events happening at transfer stops
transfer_stops_mask = stop_times_with_route.stop_id.isin(transfer_stops)


# This is: `stop_times` table, but:
# - Only transfer stops
# - Sorted by stop_id and arrival_time

subset = stop_times_with_route.loc[transfer_stops_mask].copy()

subset["arrival_event"] = (
    subset["stop_id"].astype(str) + "-" + subset["trip_id"].astype(str) + "-" + "arrival"
)
subset["departure_event"] = (
    subset["stop_id"].astype(str) + "-" + subset["trip_id"].astype(str) + "-" + "departure"
)

stop_times_by_stop_in_time_seq = subset.sort_values(
    ["stop_id", "arrival_time"]
).copy()

In [13]:
stop_times_by_stop_in_time_seq.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,route_id,arrival_event,departure_event
1576675,414728,0 days 04:48:00,0 days 04:48:00,2,23,9,2-414728-arrival,2-414728-departure
1744261,421851,0 days 05:02:00,0 days 05:02:00,2,39,65,2-421851-arrival,2-421851-departure
1386396,409639,0 days 05:11:00,0 days 05:11:00,2,39,65,2-409639-arrival,2-409639-departure
1391294,409715,0 days 05:12:00,0 days 05:12:00,2,39,65,2-409715-arrival,2-409715-departure
1575117,414699,0 days 05:12:00,0 days 05:12:00,2,23,9,2-414699-arrival,2-414699-departure


Then, add transfer edges where:
1. The stop is the same
2. The arrival time of the previous trip is 0-1800 seconds before the departure time of the next trip
3. To a different route

In [14]:
transfer_stop = 2

In [15]:
# Test: All the transfer edges happening at stop 2
stop_times_at_stop = subset.query("stop_id == @transfer_stop").copy()
all_routes_at_stop = stop_times_at_stop.route_id.unique()

In [19]:
stop_times_at_stop_by_route = {}

for route in all_routes_at_stop:
    stop_times_at_stop_by_route[route] = stop_times_at_stop.query('route_id == @route').copy()

In [26]:
def get_transfer_table(from_table, to_table):
    '''
    `from_table` and `to_table` are tables of stop_times at one transfer stop for one route.
    sorted by arrival_time
    `from_table` corresponds to the route transferring FROM, and `to_table` corresponds to the route transferring TO
    '''

    # Initialize the result table
    result = pd.DataFrame(columns=["from_event", "to_event"])
    
    pointer_index = 0

    # Both the "from" table and the "to" table are sorted by stop sequence, and therefore time

    # Iterate through each arrival for the "from" route
    # and try to find a transfer for the target route

    # To find such a transfer, iterate through the second table
    # When a transfer is found, break the iteration, and a pointer is moved to this table
    # For the next departure from the "from" table, ITS transfer cannot be earlier than this one
    # Therefore, for the next departure, iterate the "to" table from this pointer


    for _, arrival in from_table.iterrows():
        arrival_time = arrival["arrival_time"]
        while pointer_index < len(to_table):
            departure = to_table.iloc[pointer_index]
            departure_time = departure["departure_time"]
            if departure_time > arrival_time:
                result = pd.concat(
                    [
                        result,
                        pd.DataFrame(
                            {
                                "from_event": [arrival["arrival_event"]],
                                "to_event": [departure["departure_event"]],
                                "weight": departure_time - arrival_time,
                            }
                        ),
                    ]
                )
                # Do not increment the pointer at this point,
                # because the next arrival might be using the same departure as its transfer
                break
            elif departure_time <= arrival_time:
                pointer_index += 1
    
    return result

In [32]:
def get_all_transfer_table_at_stop(stop_times_at_stop):
    all_routes_at_stop = stop_times_at_stop.route_id.unique()

    # Initialize the result table
    transfer_table_for_stop = pd.DataFrame()

    stop_times_at_stop_by_route = {}

    # Get all subsets for each route
    for route in all_routes_at_stop:
        stop_times_at_stop_by_route[route] = stop_times_at_stop.query('route_id == @route').copy()

    for source_route, target_route in itertools.permutations(all_routes_at_stop, 2):
        from_table = stop_times_at_stop_by_route[source_route]
        to_table = stop_times_at_stop_by_route[target_route]

        this_transfer_table = get_transfer_table(from_table, to_table)
        transfer_table_for_stop = pd.concat([transfer_table_for_stop, this_transfer_table])
    
    return transfer_table_for_stop

In [36]:
final_transfer_table = pd.DataFrame()

for stop in transfer_stops:

    stop_times_at_stop = subset.query("stop_id == @stop").copy()
    transfers_this_stop = get_all_transfer_table_at_stop(stop_times_at_stop)

    final_transfer_table = pd.concat([final_transfer_table, transfers_this_stop])

In [38]:
for _, row in final_transfer_table.iterrows():
    G_transfer_edges.add_edge(
        row["from_event"],
        row["to_event"],
        type="transfer",
        weight=row["weight"],
    )

In [37]:
final_transfer_table

,from_event,to_event,weight
0,2-392852-arrival,2-408470-departure,0 days 01:31:00
0,2-392853-arrival,2-408470-departure,0 days 02:11:00
0,2-392854-arrival,2-408470-departure,0 days 02:51:00
0,2-392855-arrival,2-408470-departure,0 days 03:31:00
0,2-392856-arrival,2-408512-departure,0 days 00:13:00
...,...,...,...
0,32637-414636-arrival,32637-397738-departure,0 days 02:34:00
0,32637-414637-arrival,32637-397738-departure,0 days 01:55:00
0,32637-414638-arrival,32637-397738-departure,0 days 01:15:00
0,32637-414639-arrival,32637-397738-departure,0 days 00:33:00


First task: make all the shapes of lines and stops

In [ ]:
shapesGrouped = gtfs["shapes"].groupby("shape_id").apply(
    lambda x: x.sort_values("shape_pt_sequence")
).reset_index(drop=True)

shapesLines = shapesGrouped.groupby("shape_id").apply(
    lambda x: LineString(zip(x["shape_pt_lon"], x["shape_pt_lat"]))
)

shapesGdf = gpd.GeoDataFrame(shapesLines, columns=["geometry"], crs="EPSG:4326")

Understand trip types. A trip type is any trip passing through the same stops in the same order

In [17]:
stopTimes = gtfs["stop_times"].sort_values(by=["trip_id", "stop_sequence"])

# Let's group trips into trip types: any trip passing through the same stops in the same order

stopTimes["stopIdStr"] = stopTimes["stop_id"].astype(str)
tripTypes = (
    stopTimes.groupby("trip_id")["stopIdStr"].apply(lambda x: "-".join(x)).reset_index()
)

In [19]:
uniqueTypes = tripTypes.groupby("stopIdStr").size().reset_index()[["stopIdStr"]]
uniqueTypes["tripTypeId"] = uniqueTypes.index

In [22]:
# Dictionary mapping each trip_id to its tripTypeId
tripTypes = tripTypes.merge(
    uniqueTypes,
    how='left',
    left_on="stopIdStr",
    right_on="stopIdStr"
)[["trip_id", "tripTypeId"]]

In [24]:
stopTimes = stopTimes.merge(
    tripTypes,
    how='left',
    left_on="trip_id",
    right_on="trip_id"
)

In [25]:
routings = stopTimes.drop_duplicates(
    subset=["tripTypeId", "stop_sequence", "stop_id"]
).sort_values(by=["tripTypeId", "stop_sequence"])[
    ["tripTypeId", "stop_sequence", "stop_id"]
].reset_index(drop=True)

In [ ]:
routings.head()

Let's create a multigraph!

## Space-expanded network

In [44]:
G = nx.MultiDiGraph()

In [45]:
# First add the nodes

for _, stop in gtfs["stops"].iterrows():
    G.add_node(
        stop["stop_id"],
        stop_name=stop["stop_name"],
        lat=stop["stop_lat"],
        lon=stop["stop_lon"],
        zoneId=stop["zone_id"],
        wheelchairBoarding=stop["wheelchair_boarding"],
    )

In [48]:
for tripType, group in routings.groupby("tripTypeId"):
    stopIds = group["stop_id"].tolist()
    for i in range(len(stopIds) - 1):
        G.add_edge(
            stopIds[i],
            stopIds[i + 1],
            key=tripType,
        )

In [ ]:
import matplotlib.pyplot as plt

# Position dictionary for each node based on lat/lon
pos = {node: (G.nodes[node]['lon'], G.nodes[node]['lat']) for node in G.nodes()}

fig, ax = plt.subplots(figsize=(10, 10))

# Draw only the edges using draw_networkx_edges
nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue', width=0.5)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('GTFS Network Graph - Edges Only')
plt.show()

## Time-expanded network